<a href="https://colab.research.google.com/github/marifinkel/G2_Grupo1Eng1111/blob/main/grupo1_eng1111_versaofinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import warnings 
warnings.filterwarnings("ignore")

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

#divisao treino e teste
from sklearn.model_selection import train_test_split

#modelos
from sklearn.neural_network import MLPClassifier

#metricas
from sklearn.metrics import accuracy_score, classification_report, roc_curve

#validacao
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("movies_metadata.csv",header=0,index_col=None)
data.drop(columns=['adult','homepage','vote_count','title','original_title','id','status','imdb_id','poster_path','spoken_languages', 'tagline','video','overview','original_language','production_companies','production_countries'],inplace=True)
data.head(10)



,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,vote_average
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,1995-10-30,373554033.0,81.0,7.7
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,1995-12-15,262797249.0,104.0,6.9
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.7129,1995-12-22,0.0,101.0,6.5
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.859495,1995-12-22,81452156.0,127.0,6.1
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",8.387519,1995-02-10,76578911.0,106.0,5.7
5,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",17.924927,1995-12-15,187436818.0,170.0,7.7
6,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",6.677277,1995-12-15,0.0,127.0,6.2
7,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",2.561161,1995-12-22,0.0,97.0,5.4
8,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",5.23158,1995-12-22,64350171.0,106.0,5.5
9,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",14.686036,1995-11-16,352194034.0,130.0,6.6


In [ ]:
data['belongs_to_collection']=data['belongs_to_collection'].notnull()
data.head()

,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,vote_average
0,True,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,1995-10-30,373554033.0,81.0,7.7
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,1995-12-15,262797249.0,104.0,6.9
2,True,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.7129,1995-12-22,0.0,101.0,6.5
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.859495,1995-12-22,81452156.0,127.0,6.1
4,True,0,"[{'id': 35, 'name': 'Comedy'}]",8.387519,1995-02-10,76578911.0,106.0,5.7


In [ ]:
data.dropna(inplace=True)

In [ ]:
def transformadata(x):
  return str(x)[0:4]

data['release_date']=data['release_date'].apply(transformadata)
data.dropna(inplace=True)
print(data['release_date'].sort_values(ascending=False))

26559    2020
33359    2018
44535    2018
38130    2018
38885    2018
         ... 
34938    1888
34933    1887
41602    1883
34937    1878
34940    1874
Name: release_date, Length: 45130, dtype: object


In [ ]:
def transformarevenue(s):
  return int(s)

data['revenue']=data['revenue'].apply(transformarevenue)
seriesrevenue=data['revenue'].loc[data['revenue']>100000]
data['revenue']=seriesrevenue


data.dropna(inplace=True)


In [ ]:
data.head()

,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,vote_average
0,True,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,1995,373554033.0,81.0,7.7
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,1995,262797249.0,104.0,6.9
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.859495,1995,81452156.0,127.0,6.1
4,True,0,"[{'id': 35, 'name': 'Comedy'}]",8.387519,1995,76578911.0,106.0,5.7
5,False,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",17.924927,1995,187436818.0,170.0,7.7


In [ ]:
seriesrev=pd.cut(data['revenue'],bins=[100000,500000,900000,2000000,5000000,10000000,20000000,50000000,
                                       80000000,100000000,300000000,500000000,data['revenue'].max()],
                labels=['100k-500k','500k-900k','900k-2m','2m-5m','5m-10m','10m-20m','20m-50m','50m-80m','80m-100m',
                        '100m-300m','300m-500m','Acima de 500m'])
data['revenue']=seriesrev
print(data['revenue'].value_counts(),'\n',data['revenue'].count())

20m-50m          1230
100m-300m         962
10m-20m           900
5m-10m            669
2m-5m             654
50m-80m           582
100k-500k         505
900k-2m           427
80m-100m          268
300m-500m         235
500k-900k         221
Acima de 500m     170
Name: revenue, dtype: int64 
 6823


In [ ]:
print(data['revenue'].value_counts())

20m-50m          1230
100m-300m         962
10m-20m           900
5m-10m            669
2m-5m             654
50m-80m           582
100k-500k         505
900k-2m           427
80m-100m          268
300m-500m         235
500k-900k         221
Acima de 500m     170
Name: revenue, dtype: int64


In [ ]:
seriesgen=data['genres'].loc[data['genres']!='[]']
data['genres']=seriesgen
data.dropna(inplace=True)

In [ ]:
srgen=data['genres'].value_counts()
srsgenero=data['genres']
srgen.head()

[{'id': 18, 'name': 'Drama'}]                                       530
[{'id': 35, 'name': 'Comedy'}]                                      450
[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]     267
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}]         200
[{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]    190
Name: genres, dtype: int64

In [ ]:
def dividegenero(x):
  if 'Comedy' in str(x):
    if 'Romance' in str(x):
      return 'Comedy and Romance'
    elif 'Drama' in str(x):
      return 'Comedy and Drama'
    elif 'Crime' in str(x):
      return 'Comedy and Crime'
    else:
      return 'Comedy'
   
  if 'Drama' in str(x):
    if 'Romance' in str(x):
     return 'Drama and Romance'
    elif 'Comedy' in str(x):
     return 'Comedy and Drama'
    elif 'Thriller' in str(x):
     return 'Drama and Thriller'
    elif 'Crime' in str(x):
      return 'Drama and Crime'
    elif 'History' in str(x):
      return 'Drama and History'
    else:
      return 'Drama'  
  elif 'Docmentary' in str(x):
    return 'Docmentary'
  elif 'Horror' in str(x):
      if 'Thriller' in str(x):
        return 'Horror and thriller'
      else:
        return 'Horror'
  elif 'Thriller' in str(x):
    if 'Horror' in str(x):
      return 'Horror and Thriller'
    if 'Action' in str(x):
      return 'Action and Thriller'
    else: 
      return 'Thriller'
  elif 'Romance' in str(x):
    if 'Comedy' in str(x):
        return 'Comedy and Romance'
    elif 'Drama' in str(x):
      return 'Drama and Romance'
    else:
      return 'Romance'

  elif 'Family' in str(x): 
    return 'Family'
  elif 'Action' in str(x):
    if 'Action' in str(x):
      return 'Action and Thriller'
    else:
      return 'Action'
  elif 'Animation' in str(x): 
    return 'Animation'
  elif 'Adventure' in str(x):
    return 'Adventure'
  elif 'Crime' in str(x):
    if 'Drama' in str(x):
      return 'Drama and Crime'
    elif 'Comedy' in str(x):
      return 'Comedy and Crime'
    else:
      return 'Crime'
  else:
    return 'others'

srgenero=srsgenero.apply(dividegenero)
print(srgenero.value_counts())



Comedy                 1098
Drama                   936
Drama and Thriller      699
Action and Thriller     697
Comedy and Romance      671
Drama and Romance       580
Comedy and Drama        473
Horror and thriller     277
Family                  198
others                  186
Horror                  182
Comedy and Crime        177
Drama and Crime         175
Thriller                165
Drama and History       165
Adventure                51
Romance                  50
Animation                26
Crime                     2
Name: genres, dtype: int64


In [ ]:
data['genres']=srgenero
data.head()

,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,vote_average
0,True,30000000,Comedy,21.946943,1995,300m-500m,81.0,7.7
1,False,65000000,Family,17.015539,1995,100m-300m,104.0,6.9
3,False,16000000,Comedy and Romance,3.859495,1995,80m-100m,127.0,6.1
4,True,0,Comedy,8.387519,1995,50m-80m,106.0,5.7
5,False,60000000,Drama and Thriller,17.924927,1995,100m-300m,170.0,7.7


In [ ]:
def transformapop(s):
  return float(s)

seriespop=data['popularity'].apply(transformapop)
data['popularity']=seriespop

In [ ]:
def transformabudget (s):
  return int(s)

seriesbudget1=data['budget'].apply(transformabudget)
data['budget']=seriesbudget1

seriesbudget2=data['budget'].loc[data['budget']>100000]

data['budget']=seriesbudget2

#print(seriesbudget.sort_values(ascending=True))



In [ ]:
data.nunique()


belongs_to_collection       2
budget                    622
genres                     19
popularity               6803
release_date               99
revenue                    12
runtime                   163
vote_average               74
dtype: int64

In [ ]:
data.dropna(inplace=True)

In [ ]:

data.head(100)

,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,vote_average
0,True,30000000.0,Comedy,21.946943,1995,300m-500m,81.0,7.7
1,False,65000000.0,Family,17.015539,1995,100m-300m,104.0,6.9
3,False,16000000.0,Comedy and Romance,3.859495,1995,80m-100m,127.0,6.1
5,False,60000000.0,Drama and Thriller,17.924927,1995,100m-300m,170.0,7.7
8,False,35000000.0,Action and Thriller,5.231580,1995,50m-80m,106.0,5.5
...,...,...,...,...,...,...,...,...
244,False,10500000.0,Comedy,2.688562,1995,20m-50m,113.0,6.0
249,True,60000000.0,Horror,11.970205,1994,100m-300m,123.0,7.2
250,False,14000000.0,Drama and Romance,1.596058,1995,2m-5m,139.0,5.2
252,False,60000000.0,Comedy and Romance,6.710215,1994,20m-50m,109.0,4.7


In [ ]:
data.isnull().sum()


belongs_to_collection    0
budget                   0
genres                   0
popularity               0
release_date             0
revenue                  0
runtime                  0
vote_average             0
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5111 entries, 0 to 45422
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   belongs_to_collection  5111 non-null   bool    
 1   budget                 5111 non-null   float64 
 2   genres                 5111 non-null   object  
 3   popularity             5111 non-null   float64 
 4   release_date           5111 non-null   object  
 5   revenue                5111 non-null   category
 6   runtime                5111 non-null   float64 
 7   vote_average           5111 non-null   float64 
dtypes: bool(1), category(1), float64(4), object(2)
memory usage: 289.9+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder

le_revenue = LabelEncoder()
data['revenue'] = le_revenue.fit_transform(data['revenue'])

In [ ]:
data['budget'] = le_revenue.fit_transform(data['budget'])

In [ ]:
data['popularity'] = le_revenue.fit_transform(data['popularity'])

In [ ]:
data['runtime'] = le_revenue.fit_transform(data['runtime'])

In [ ]:
data['vote_average'] = le_revenue.fit_transform(data['vote_average'])

In [ ]:
data['genres'] = le_revenue.fit_transform(data['genres'])

In [ ]:
data['belongs_to_collection'] = le_revenue.fit_transform(data['belongs_to_collection'])

In [ ]:
data['release_date'] = le_revenue.fit_transform(data['release_date'])

In [ ]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5111 entries, 0 to 45422
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   belongs_to_collection  5111 non-null   int64
 1   budget                 5111 non-null   int64
 2   genres                 5111 non-null   int64
 3   popularity             5111 non-null   int64
 4   release_date           5111 non-null   int64
 5   revenue                5111 non-null   int64
 6   runtime                5111 non-null   int64
 7   vote_average           5111 non-null   int64
dtypes: int64(8)
memory usage: 359.4 KB


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
data[['belongs_to_collection','budget','genres', 'popularity','release_date', 'runtime', 'vote_average']] = sc.fit_transform(data[['belongs_to_collection','budget','genres', 'popularity', 'release_date','runtime', 'vote_average']])

In [ ]:
data.head()

,belongs_to_collection,budget,genres,popularity,release_date,revenue,runtime,vote_average
0,1.804334,0.546650,-0.919776,1.609823,-0.306533,5,-1.405954,1.588569
1,-0.554221,0.991918,1.108254,1.469483,-0.306533,1,-0.308239,0.685071
3,-0.554221,0.166018,-0.311367,-1.215964,-0.306533,9,0.789475,-0.218428
5,-0.554221,0.934464,0.905451,1.505415,-0.306533,1,2.841724,1.588569
8,-0.554221,0.632831,-1.528185,-0.996979,-0.306533,7,-0.212786,-0.896052


In [ ]:
data.shape

(5111, 8)

In [ ]:
y = data['revenue']
X = data.drop('revenue', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
print("Shape X_train", X_train.shape)
print("Shape X_test",X_test.shape)
print("Shape y_train",y_train.shape)
print("Shape y_test",y_test.shape)

Shape X_train (4088, 7)
Shape X_test (1023, 7)
Shape y_train (4088,)
Shape y_test (1023,)


In [ ]:
from sklearn.neural_network import MLPClassifier
X = X_train
y = y_train
clf= MLPClassifier(hidden_layer_sizes=(150,200,250), max_iter=300,activation = 'relu',solver='lbfgs',random_state=1,alpha=1e-5)

clf.fit(X, y)
MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='lbfgs',random_state=1,alpha=1e-5)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(150, 100, 50), max_iter=300,
              random_state=1, solver='lbfgs')

In [ ]:
X

,belongs_to_collection,budget,genres,popularity,release_date,runtime,vote_average
7832,-0.554221,0.661558,-0.919776,-0.285787,0.268657,-0.069606,0.007447
16288,-0.554221,0.546650,-0.514170,-0.348838,0.652117,1.123562,1.023882
758,1.804334,-1.004607,0.297042,1.557619,-0.242623,-0.833233,1.701506
11499,-0.554221,0.546650,0.702648,0.002351,0.396477,-0.880960,0.007447
6361,-0.554221,-1.363694,0.702648,-1.412576,-0.434353,-0.737780,-2.251299
...,...,...,...,...,...,...,...
40497,1.804334,-0.825063,1.513860,0.866766,1.035577,-0.976413,-1.573676
13800,-0.554221,-1.945415,0.297042,-0.188159,-1.712552,-0.165059,0.459196
4844,-0.554221,0.166018,-0.919776,0.135912,0.076927,-1.024140,-0.896052
10545,-0.554221,0.525105,0.702648,1.272871,0.332567,1.171289,1.701506


In [ ]:

y_predict = clf.predict(X)
y_predict


array([1, 3, 2, ..., 7, 1, 8])

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = y_predict
y_true = y_train

print(f"A precisao do teste usando Metrics foi de {accuracy_score(y_true, y_pred,normalize=True)*100:.2f}% ")

A precisao do teste usando Metrics foi de 82.44% 
